In [1]:
from ase.io import read
from lammps import lammps
import os
import sys
import numpy as np
sys.path.append("/home/dux/")
sys.path.append("/home/dux/surface_sampling/sgmc_surf")

from htvs.djangochem.pgmols.utils import surfaces

os.environ["LAMMPS_COMMAND"] = "/home/dux/lammps/src/lmp_serial"
os.environ["LAMMPS_POTENTIALS"] = "/home/dux/lammps/potentials/"

main_dir = "/home/dux/surface_sampling/sgmc_surf/GaN"

# define necessary file locations
lammps_in_file = f"{main_dir}/lammps.in"
potential_file = f"GaN.tersoff"
atoms = ["Ga", "N"]
lammps_out_file = f"{main_dir}/lammps.out"
cif_from_lammps_file = f"{main_dir}/lammps_from_cif.cif"

In [2]:
supercell_atoms = read('GaN_hexagonal.cif')*(3,3,3)
supercell_atoms.write('GaN_hexagonal_3x3.cif')

bulk_slab, surface_atoms = surfaces.surface_from_bulk(supercell_atoms, [0,0,0,-1], size=[3,3], vacuum=10)

# REMOVE THESE COMPLICATED STUFF FIRST
# set surface atoms from the other side
all_atoms = np.arange(len(bulk_slab))
curr_surf_atoms = bulk_slab.get_surface_atoms()
new_surf_atoms = np.setdiff1d(all_atoms, curr_surf_atoms)
bulk_slab.set_surface_atoms(new_surf_atoms)
# invert the positions
bulk_slab.set_scaled_positions(1 - bulk_slab.get_scaled_positions())
bulk_slab.write("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_with_scaled_postions_inverted.cif")

lammps_data_file = f"{main_dir}/lammps_from_cell.data"
bulk_slab.write(
    lammps_data_file, format="lammps-data", units="metal", atom_style="atomic"
)

cell = bulk_slab.get_cell()
print(cell)

correct_cell = cell

/home/dux/miniconda3/envs/surface_sampling/lib/python3.8/site-packages/catkit/gen/surface.py:334: UserWarning: Your slab has no bulk atoms and may be too thin to identify surface atoms correctly. This may cause surface adsorption site identification to fail.
  warnings.warn(


Cell([[9.648870209999998, 0.0, 0.0], [4.824435104999999, 8.35616671967889, 0.0], [0.0, 0.0, 23.26575391688]])


In [3]:
lammps_data_file = f"{main_dir}/lammps_from_cif.data"
cif_from_cif_file = f"{main_dir}/cif_from_cif.cif"

# cif_slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_with_scaled_postions.cif")
# cif_slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_with_scaled_postions_inverted.cif")
cif_slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_3x3_pymatgen/runs500_temp2.0_adsatoms12_alpha0.99_20220527-183550/optim_slab_run_500.cif")
cif_slab.cell = correct_cell
cif_slab.write(
    lammps_data_file, format="lammps-data", units="metal", atom_style="atomic"
)
cif_slab.write(cif_from_cif_file)

In [4]:
# write current surface into lammps.data
slab = cif_slab
slab.write(cif_from_lammps_file)
TEMPLATE = """
clear
atom_style atomic
units metal
boundary p p p
atom_modify sort 0 0.0

# read_data /path/to/data.data
read_data {}

### set bulk
group bulk id <= 36
group surface id > 36

### interactions
pair_style tersoff
# pair_coeff * * /path/to/potential Atom1 Atom2
pair_coeff * * {} {} {}
mass 1 69.723000
mass 2 14.007000

### run
reset_timestep 0
velocity surface create 100.0 87287 dist gaussian
fix 1 all nvt temp 100.0 0.1 $(100.0*dt)

# set bulk forces and velocities to 0
fix 2 bulk setforce 0.0 0.0 0.0
velocity bulk set 0.0 0.0 0.0

thermo 10 # output thermodynamic variables every N timesteps

thermo_style custom step temp press ke pe xy xz yz
thermo_modify flush yes format float %23.16g
# min_style cg
# minimize 1e-5 1e-5 500 10000

timestep 0.001
run 3000

# write_data /path/to/data.out
write_data {}
print "_end of MD_"
log /dev/stdout

"""

# write lammps.in file
with open(lammps_in_file, "w") as f:
    f.writelines(
        TEMPLATE.format(lammps_data_file, potential_file, *atoms, lammps_out_file)
    )

lmp = lammps()
# print("LAMMPS Version: ", lmp.version())

# run the LAMMPS here
lmp.file(lammps_in_file)
lmp.close()

# Read from LAMMPS out
opt_slab = read(lammps_out_file, format="lammps-data", style="atomic")

atomic_numbers_dict = {1: 31, 2: 7}  # 1: Ga, 2: N
actual_atomic_numbers = [
    atomic_numbers_dict[x] for x in opt_slab.get_atomic_numbers()
]
print(f"actual atomic numbers {actual_atomic_numbers}")
# correct_lammps = new_slab.copy()
opt_slab.set_atomic_numbers(actual_atomic_numbers)
opt_slab.calc = slab.calc
opt_slab.write(cif_from_lammps_file)

LAMMPS (29 Sep 2021)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0.0000000 0.0000000 0.0000000) to (9.6488702 8.3561667 23.265754) with tilt (4.8244351 0.0000000 0.0000000)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  48 atoms
  read_data CPU = 0.000 seconds
36 atoms in group bulk
12 atoms in group surface
Reading tersoff potential file GaN.tersoff with DATE: 2007-10-22
Neighbor list info ...
  update every 1 steps, delay 10 steps, check yes
  max neighbors/atom: 2000, page size: 100000
  master list distance cutoff = 5.1
  ghost atom cutoff = 5.1
  binsize = 2.55, bins = 6 4 10
  1 neighbor lists, perpetual/occasional/extra = 1 0 0
  (1) pair tersoff, perpetual
      attributes: full, newton on
      pair build: full/bin/atomonly
      st